# ID2222 Data Mining, Homework 3
# **Mining Data Streams**

Brando Chiminelli, Tommaso Praturlon

November 28th, 2022

## Goal
The goal of this notebook is to 

## How to run

In order to run this notebook you need to 

## Import libraries and read the dataset
In the following we import the few libraries needed for the project and we read the dataset.

We decided to read the first 2000 baskets from the dataset in order to reduce weight on memory. Our assumption is that items are uniformly distributed across the dataset, thus allowing us to have a good insight only from the given sample.

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
import statistics
import math

PATH_TO_DATA = "../data/web-Stanford.txt"
df_graphs = pd.read_csv(PATH_TO_DATA, header=None)
print("Data read successfully!")

# Reduce dataset size for computation overload (temporary)
#df_graphs = df_graphs.iloc[0:50000]
print(df_graphs.head())
print("Number of rows: ", len(df_graphs))

Data read successfully!
                                                   0
0  # Directed graph (each unordered pair of nodes...
1                     # Stanford web graph from 2002
2                     # Nodes: 281903 Edges: 2312497
3                             # FromNodeId\tToNodeId
4                                            1\t6548
Number of rows:  2312501


In [13]:
# DATA WRANGLING
# 1. remove text rows from dataset
# 2. create a dataset of integers
df_g = df_graphs.drop([0, 1, 2, 3, 4])
data = []
for i in range(len(df_g)):
    s = [int(x) for x in str(df_g.iloc[i][0]).split('\t')]
    #s = str(df_g.iloc[i][0]).split('\t')
    data.append(s)
#print(data)

In [14]:
####################################
# TRIEST-BASE CLASS IMPLEMENTATION #
####################################

class TriestBase:
    '''
    Implementation of the Trièst-base algorithm
    - function SampleEdge
    - function UpdateCounter
    - function FlipBiasedCoin
    '''
    
    #M = 6
    
    def flipBiasedCoin(self, M, t):
        '''
        Flip a biased coin with probability M/t of falling head.
        '''
        import numpy as np
        # 1: head, 0: tail
        result = np.random.choice([1, 0], p=[M/t, (1-M/t)])
        if result:
            return True
        else:
            return False
    
    def sampleEdge(self, edge, t, S, M, tau, tau_loc):
        '''
        Receives the edge (u,v) as [u, v] and time t at which
        the stream element is received. t is a count integer.
        Returns a boolean
        '''
        import random
        
        if (t <= M):
            return True
        elif self.flipBiasedCoin(M, t):
            # select random edge from S
            random_edge = random.choice(S)
            # Delete random_edge from S
            S.remove(random_edge)
            # Update counters
            self.updateCounters('delete', random_edge, S, tau, tau_loc)
            return True
        else:
            return False
            
    def updateCounters(self, operation, edge, S, tau, tau_loc):
        '''
        Receives the operation insertion or deletion
        and the edge.
        tau is the global counter
        e.g S = [[5,10], [10, 3], [12, 5], [12, 3]]
        edge = (5, 12)
        N_5 = (10, 12)
        N_12 = (5, 3)
        N_5_12 = (5, 12)
        '''
        # Define shared-neighborhood
        shared_neigh = set()
        neigh_u = set() # all neighbors of edge[0]
        neigh_v = set() # all nneighbors of edge[1]
        #print("S from updateCounters():\n", S)
        for elem in S:
            # check that for the v in V_t (u,v) belongs to S
            # create neigh_u
            if edge[0] == elem[0]: # found u in position 0
                neigh_u.add(edge[1]) # add the other element
            if edge[0] == elem[1]: # found u in position 1
                neigh_u.add(edge[0]) # add the other element
            # create neigh_v
            if edge[1] == elem[0]: # found v in position 0
                neigh_v.add(edge[1]) 
            if edge[1] == elem[1]: # found v in position 1
                neigh_v.add(edge[0])
        # shared neighbourhood is the intersection between the sets
        shared_neigh = set.intersection(neigh_u, neigh_v)
            
        if operation == 'insert':
            # Insert
            for c in shared_neigh:
                tau += 1
                tau_loc[c] = tau_loc.get(c, 0) + 1
                tau_loc[edge[0]] = tau_loc.get(edge[0], 0) + 1
                tau_loc[edge[1]] = tau_loc.get(edge[1], 0) + 1
        else:
            # Delete
            for c in shared_neigh:
                tau -= 1
                tau_loc[c] = tau_loc.get(c, 0) - 1
                if tau_loc[c] <= 0:
                    del tau_loc[c]
                tau_loc[edge[0]] = tau_loc.get(edge[0], 0) - 1
                if tau_loc[edge[0]] <= 0:
                    del tau_loc[edge[0]]
                tau_loc[edge[1]] = tau_loc.get(edge[1], 0) - 1
                if tau_loc[edge[1]] <= 0:
                    del tau_loc[edge[1]]
                    
        return S, tau, tau_loc

In [15]:
# EXPLOITING THE CLASS TRIEST-BASE

'''
At time _t
Graph G_t = (set of verteces V_t, set of edges E_t) 
'''

S = []
M = 242238
t = 0
tau = 0
tau_loc = {} # dictionary of local counts (u: counts) {5: 99, 12: 3, 2:1}

print("Number of edges: ", len(data))
# simulate the stream of data
for edge in data:
    t += 1
    #print(edge)
    if TriestBase().sampleEdge(edge, t, S, M, tau, tau_loc):
        S.append(edge)
        #print(S)
        S, tau, tau_loc = TriestBase().updateCounters('insert', edge, S, tau, tau_loc)
        #print("Tau: ", tau)
        
# Epsilon
print(t)
eps = (t*(t-1)*(t-2))/(M*(M-1)*(M-2))
est_triangles = eps*tau
print("Eps: ", eps)
print("Tau: ", tau)
print("Estimated triangles: ", est_triangles)

Number of edges:  2312496


KeyboardInterrupt: 

In [7]:
# Graphs and statistics 
import matplotlib.pyplot as plt
from collections import Counter
'''
cnt = Counter(C_1.values())
plt.bar(cnt.keys(), cnt.values())

plt.xlabel('Frequency')
plt.ylabel('Number of documents with that frequency')
plt.show()
'''

"\ncnt = Counter(C_1.values())\nplt.bar(cnt.keys(), cnt.values())\n\nplt.xlabel('Frequency')\nplt.ylabel('Number of documents with that frequency')\nplt.show()\n"